In [2]:
import tkinter
from tkinter import *
from tkinter.ttk import *
from PIL import ImageTk, Image

import numpy as np
from matplotlib import pyplot as plt
from scipy import misc
import matplotlib as mpl

import os.path

# colormap
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

# Mandelbrot computation
def compute_mandelbrot(num_iters, res,left_R,right_R,down_I,up_I, file_name = 'Mandelbrot'):
    dt = np.dtype(np.complex128)
    x = np.linspace(left_R,right_R,res,dt)
    y = np.linspace(up_I*1j,down_I*1j,res,dt)
    B = x + y.reshape(res,1)
    z = np.zeros((res,res),dt)
    dz = np.ones((res,res),dt)
    n = 0
    overflow_threshold = 10000000.
    while n < num_iters:
        abs_z = np.absolute(z)
        abs_dz = np.absolute(dz)
        safe = (abs_z < overflow_threshold) & (abs_dz < overflow_threshold)
        dz[safe] = 2*z[safe]*dz[safe] + 1
        z[safe] = z[safe]**2 + B[safe]
        n += 1
    M = (abs_z*np.log(abs_z)/abs_dz)
    Mand = (abs_z < 2)
    M[Mand] = 0
    inds = np.where(np.isnan(M))
    M[inds]=np.nanmax(M)
            
    cmap_cl = mpl.cm.rainbow
    bins = np.zeros((22,),dtype = np.float64)
    bins[1:] = np.percentile(M[M > 0.], np.arange(0.,101.,100./20.))
    norm = mpl.colors.BoundaryNorm(boundaries=bins, ncolors = 256)
    
    image_cl = cmap_cl(norm(M))
    
    gray_cmap = mpl.cm.gray
    cmap_BW = truncate_colormap(gray_cmap, 0.2, 1)
    
    image_BW = cmap_BW(norm(M))
    
    image_cl[Mand, :3] = 0,0,0
    
    image_BW[Mand] = 0
    file_name_cl = file_name + '_colour.jpg'
    file_name_BW = file_name + '_BW.jpg'
    
    misc.imsave(file_name_cl,image_cl)
    misc.imsave(file_name_BW,image_BW)

# Color-BW Mandelbrot    
def regular_mandelbrot():
    
    compute_mandelbrot(100, 820,-2,1,-1.5,1.5, 'Mandelbrot')
    
# x3 zoom-in Mandelbrot
def zoom_x3_mandelbrot(c_R,c_I):
    
    compute_mandelbrot(100,820,c_R-0.5,c_R+0.5,c_I-0.5,c_I+0.5, 'Mandelbrot_zoom_x3')


def zoom_x9_mandelbrot(c_R,c_I):
    
    compute_mandelbrot(100,820,c_R-0.17,c_R+0.17,c_I-0.17,c_I+0.17,'Mandelbrot_zoom_x9')

    
def zoom_x27_mandelbrot(c_R,c_I):
    
    compute_mandelbrot(100,820,c_R-0.06,c_R+0.06,c_I-0.06,c_I+0.06,'Mandelbrot_zoom_x27')

# Julia computation
def compute_Julia_set(num_iters, res, c,left_R,right_R,down_I,up_I, file_name = 'Julia_set'):
    dt = np.dtype(np.complex128)
    x = np.linspace(left_R,right_R,res,dt)
    y = np.linspace(up_I*1j,down_I*1j,res,dt)
    z = x + y.reshape(res,1)
    dz = np.ones((res,res),dt)
    n = 0
    overflow_threshold = 10000000.
    while n < num_iters:
        abs_z = np.absolute(z)
        abs_dz = np.absolute(dz)
        safe = (abs_z < overflow_threshold) & (abs_dz < overflow_threshold)
        dz[safe] = 2*z[safe]*dz[safe]
        z[safe] = z[safe]**2 + c
        n += 1
    M = (abs_z*np.log(abs_z)/abs_dz)
    Julia = (abs_z < 2)
    cmap_cl = mpl.cm.rainbow
    bins = np.zeros((22,),dtype = np.float64)
    bins[1:] = np.percentile(M[M > 0.], np.arange(0.,101.,100./20.))
    norm = mpl.colors.BoundaryNorm(boundaries=bins, ncolors = 256)
    
    image_cl = cmap_cl(norm(M))
    
    gray_cmap = mpl.cm.gray
    cmap_BW = truncate_colormap(gray_cmap, 0.2, 1)
    
    image_BW = cmap_BW(norm(M))
    
    image_cl[Julia, :3] = 0,0,0
    
    image_BW[Julia] = 0
    file_name_cl = file_name + '_colour.jpg'
    file_name_BW = file_name + '_BW.jpg'
    
    misc.imsave(file_name_cl,image_cl)
    misc.imsave(file_name_BW,image_BW)
    
# Color-BW Julia
def regular_Julia_set(c):
    
    compute_Julia_set(200,820,c,-2,2,-2,2, 'Julia_set')
    
# x3 zoom-in Julia
def zoom_x3_Julia_set(c,p_R,p_I):
    
    compute_Julia_set(200,820,c,p_R-0.67,p_R+0.67,p_I-0.67,p_I+0.67, 'Julia_set_zoom_x3')
    

def zoom_x9_Julia_set(c,p_R,p_I):
    
    compute_Julia_set(200,820,c,p_R-0.22,p_R+0.22,p_I-0.22,p_I+0.22,'Julia_set_zoom_x9')    
    

def zoom_x27_Julia_set(c,p_R,p_I):
    
    compute_Julia_set(200,820,c,p_R-0.07,p_R+0.07,p_I-0.07,p_I+0.07,'Julia_set_zoom_x27')    
    
    
mandel_or_julia = True  #This shall be used in the switching button Mandelbrot/Julia
alter_bw_color = True   #This shall be used in the switching button for BW/colored image
zoom_level = 1
center_x_3 = 0
center_y_3 = 0
center_x_9 = 0
center_y_9 = 0
    

    
# GUI
class Application(tkinter.Frame):
    
    def __init__(self, master=None):
        self.root = root 
        super().__init__(master)
        
        self.create_button_widgets()
        self.c_entry()
        
        self.main_container = Frame(master)
        self.main_container.grid(row=0,column=0, sticky="NSEW")
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)
        
        # Create a canvas
        self.canvas = tkinter.Canvas(self.root,width=500,height=500)
        self.canvas.grid(column=2, row=1, sticky=NE, columnspan=8, rowspan=8) 
        self.canvas.bind('<Button-1>', self.zoom)
        self.canvas.bind('<Button-3>', self.right_click)
        
        
        
    def create_button_widgets(self):
        # Here we create some buttons.
        
        # The Mandelbrot/Julia set button
        self.Mandelbrot_Julia = tkinter.Button(self.root)
        self.Mandelbrot_Julia["text"] = "Mandelbrot/Julia Set"
        self.Mandelbrot_Julia["command"] = self.open_image
        self.Mandelbrot_Julia.grid(column=0, row=1, sticky=W)
        
        # The BW/color button
        self.BW_color = tkinter.Button(self.root)
        self.BW_color["text"] = "BW/color"
        self.BW_color["command"] = self.color_BW
        self.BW_color.grid(column=1, row=1, sticky=W)
        
        # Label for c parameter
        label = tkinter.Label(self.root, text = "c-Parameter :")
        label.grid(column=0, row=2, sticky=W)
        
        # Entry of c parameter
        c = StringVar() 
        self.entry = Entry(self.root, textvariable=c)
        self.entry.grid(column=0, row=3, sticky=W)
        
        # Button to calculate the Julia set
        self.calculate_Julia = tkinter.Button(self.root, text="Calculate Julia set" , command=self.calcul_Julia)
        self.calculate_Julia.grid(column=1, row=3, sticky=W)
        
        # The zoom-out button
        self.zoom_out = tkinter.Button(self.root, text="zoom out", command = self.zoom_out)
        self.zoom_out.grid(column=0,row=4, sticky=W)
        
        # Reset button
        self.reset = tkinter.Button(self.root, text="reset", command = self.reset) #command should be "self.the_reset_function"
        self.reset.grid(column=1, row=4, sticky=W)
        
        
        # The quit button      
        self.quit = tkinter.Button(self.root, text="QUIT", fg="red",  command=root.destroy)
        self.quit.grid(column=0,row=5, sticky=W)  
        
    def open_image(self):
        global mandel_or_julia
        global zoom_level
        
        if mandel_or_julia == True :
            if alter_bw_color == False :
                image = Image.open('Mandelbrot_colour.jpg')
            elif alter_bw_color == True :
                image = Image.open('Mandelbrot_BW.jpg')
            # Image resize
            image = image.resize((410,410), Image.ANTIALIAS)
            self.image = ImageTk.PhotoImage(image)
            self.canvas.create_image(250, 250, image=self.image)
            mandel_or_julia = False
        else :
            if alter_bw_color == False :
                image = Image.open('Julia_set_colour.jpg')
            elif alter_bw_color == True :
                image = Image.open('Julia_set_BW.jpg')
            # Image resize
            image = image.resize((410,410), Image.ANTIALIAS)
            self.image = ImageTk.PhotoImage(image)
            self.canvas.create_image(250, 250, image=self.image)
            mandel_or_julia = True

        zoom_level = 1
        
    def c_entry(self):
        c = self.entry.get()
        return c
        
    
    def calcul_Julia(self):
        global mandel_or_julia
        c = self.c_entry()
        c_num = complex(c)
        mandel_or_julia = True
        regular_Julia_set(c_num)
        if alter_bw_color == False :
            image = Image.open('Julia_set_colour.jpg')
        elif alter_bw_color == True :
            image = Image.open('Julia_set_BW.jpg')
        # Image resize
        image = image.resize((410,410), Image.ANTIALIAS)
        self.image = ImageTk.PhotoImage(image)
        self.canvas.create_image(250, 250, image=self.image)

    
    
    def color_BW(self):
        global mandel_or_julia
        global alter_bw_color
        global zoom_level
        
        if mandel_or_julia == False :
            if alter_bw_color == True :
                if zoom_level == 1:
                    image = Image.open('Mandelbrot_colour.jpg')
                elif zoom_level == 3:
                    image = Image.open('Mandelbrot_zoom_x3_colour.jpg')
                elif zoom_level == 9:
                    image = Image.open('Mandelbrot_zoom_x9_colour.jpg')
                elif zoom_level == 27:
                    image = Image.open('Mandelbrot_zoom_x27_colour.jpg')
                # Image resize
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                alter_bw_color = False
            else :
                if zoom_level == 1:
                    image = Image.open('Mandelbrot_BW.jpg')
                elif zoom_level == 3:
                    image = Image.open('Mandelbrot_zoom_x3_BW.jpg')
                elif zoom_level == 9:
                    image = Image.open('Mandelbrot_zoom_x9_BW.jpg')
                elif zoom_level == 27:
                    image = Image.open('Mandelbrot_zoom_x27_BW.jpg')
                # Image resize
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                alter_bw_color = True
            
        else :
            if alter_bw_color == True :
                if zoom_level == 1:
                    image = Image.open('Julia_set_colour.jpg')
                elif zoom_level == 3:
                    image = Image.open('Julia_set_zoom_x3_colour.jpg')
                elif zoom_level == 9:
                    image = Image.open('Julia_set_zoom_x9_colour.jpg')
                elif zoom_level == 27:
                    image = Image.open('Julia_set_zoom_x27_colour.jpg')
                # Image resize
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                alter_bw_color = False 
            else :
                if zoom_level == 1:
                    image = Image.open('Julia_set_BW.jpg')
                elif zoom_level == 3:
                    image = Image.open('Julia_set_zoom_x3_BW.jpg')
                elif zoom_level == 9:
                    image = Image.open('Julia_set_zoom_x9_BW.jpg')
                elif zoom_level == 27:
                    image = Image.open('Julia_set_zoom_x27_BW.jpg')
                # Image resize
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                alter_bw_color = True 
    
    def zoom(self,event):
        global mandel_or_julia
        global alter_bw_color
        global zoom_level
        global center_x_3
        global center_y_3        
        global center_x_9
        global center_y_9
        
        if mandel_or_julia == False :
            if zoom_level == 1:
                x,y = (event.x-45)*3/410 - 2,(45-event.y)*3/410 + 1.5
                zoom_x3_mandelbrot(x,y)
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_zoom_x3_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_zoom_x3_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                center_x_3, center_y_3 = x ,y
                zoom_level = 3
            
            elif zoom_level == 3:
                x,y = (event.x-45)/410 + center_x_3-0.5,(45-event.y)/410 +center_y_3 + 0.5
                zoom_x9_mandelbrot(x,y)
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_zoom_x9_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_zoom_x9_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                center_x_9, center_y_9 = x ,y
                zoom_level = 9
            
            elif zoom_level == 9:
                x,y = (event.x-45)*0.34/410 + center_x_9-0.17,(45-event.y)*0.34/410 +center_y_9 + 0.17
                zoom_x27_mandelbrot(x,y)
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_zoom_x27_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_zoom_x27_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                zoom_level = 27
        
        elif mandel_or_julia == True :
            if zoom_level == 1:
                x,y = (event.x-45)*4/410 - 2,(45-event.y)*4/410 + 2
                c = self.c_entry()
                c_num = complex(c)
                zoom_x3_Julia_set(c_num,x,y)
                if alter_bw_color == False :
                    image = Image.open('Julia_set_zoom_x3_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_zoom_x3_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                center_x_3, center_y_3 = x ,y
                zoom_level = 3
            
            elif zoom_level == 3:
                x,y = (event.x-45)*1.34/410 + center_x_3 - 0.67, (45-event.y)*1.34/410 + center_y_3 + 0.67
                c = self.c_entry()
                c_num = complex(c)
                zoom_x9_Julia_set(c_num,x,y)
                if alter_bw_color == False :
                    image = Image.open('Julia_set_zoom_x9_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_zoom_x9_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                center_x_9, center_y_9 = x ,y
                zoom_level = 9
            
            
            elif zoom_level == 9:
                x,y = (event.x-45)*0.44/410 + center_x_9 - 0.22, (45-event.y)*0.44/410 + center_y_9 + 0.22
                c = self.c_entry()
                c_num = complex(c)
                zoom_x27_Julia_set(c_num,x,y)
                if alter_bw_color == False :
                    image = Image.open('Julia_set_zoom_x27_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_zoom_x27_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                zoom_level = 27
    
    
    def zoom_out(self):
        global mandel_or_julia
        global alter_bw_color
        global zoom_level
        
        if zoom_level == 3:
            if mandel_or_julia == False :
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                
            elif mandel_or_julia == True :
                if alter_bw_color == False :
                    image = Image.open('Julia_set_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
            zoom_level = 1
        
        if zoom_level == 9:
            if mandel_or_julia == False :
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_zoom_x3_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_zoom_x3_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                
            elif mandel_or_julia == True :
                if alter_bw_color == False :
                    image = Image.open('Julia_set_zoom_x3_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_zoom_x3_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
            zoom_level = 3
        
        if zoom_level == 27:
            if mandel_or_julia == False :
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_zoom_x9_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_zoom_x9_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                
            elif mandel_or_julia == True :
                if alter_bw_color == False :
                    image = Image.open('Julia_set_zoom_x9_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_zoom_x9_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
            zoom_level = 9
        
    
    def right_click(self,event):
        global mandel_or_julia
        global alter_bw_color
        global zoom_level
        if zoom_level == 3:
            if mandel_or_julia == False :
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                
            elif mandel_or_julia == True :
                if alter_bw_color == False :
                    image = Image.open('Julia_set_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
            zoom_level = 1
        
        if zoom_level == 9:
            if mandel_or_julia == False :
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_zoom_x3_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_zoom_x3_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                
            elif mandel_or_julia == True :
                if alter_bw_color == False :
                    image = Image.open('Julia_set_zoom_x3_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_zoom_x3_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
            zoom_level = 3
        
        if zoom_level == 27:
            if mandel_or_julia == False :
                if alter_bw_color == False :
                    image = Image.open('Mandelbrot_zoom_x9_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Mandelbrot_zoom_x9_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
                
            elif mandel_or_julia == True :
                if alter_bw_color == False :
                    image = Image.open('Julia_set_zoom_x9_colour.jpg')
                elif alter_bw_color == True :
                    image = Image.open('Julia_set_zoom_x9_BW.jpg')
                image = image.resize((410,410), Image.ANTIALIAS)
                self.image = ImageTk.PhotoImage(image)
                self.canvas.create_image(250, 250, image=self.image)
            zoom_level = 9
        
    def reset(self):
        global mandel_or_julia
        global alter_bw_color
        global zoom_level
        if mandel_or_julia == False :
            if alter_bw_color == False :
                image = Image.open('Mandelbrot_colour.jpg')
            elif alter_bw_color == True :
                image = Image.open('Mandelbrot_BW.jpg')
            image = image.resize((410,410), Image.ANTIALIAS)
            self.image = ImageTk.PhotoImage(image)
            self.canvas.create_image(250, 250, image=self.image)
                
        elif mandel_or_julia == True :
            if alter_bw_color == False :
                image = Image.open('Julia_set_colour.jpg')
            elif alter_bw_color == True :
                image = Image.open('Julia_set_BW.jpg')
            image = image.resize((410,410), Image.ANTIALIAS)
            self.image = ImageTk.PhotoImage(image)
            self.canvas.create_image(250, 250, image=self.image)
        zoom_level = 1
        
        
            
#creating the color images
if os.path.isfile('Mandelbrot_BW.jpg') == False & os.path.isfile('Mandelbrot_colour.jpg') == False:
    regular_mandelbrot()
if os.path.isfile('Julia_set_BW.jpg') == False & os.path.isfile('Julia_set_colour.jpg') == False:
    regular_Julia_set(0)
      
        
if __name__ == '__main__' :
    root = tkinter.Tk()
    app = Application(master=root)
    app.mainloop()